# Preporcessing data -train

# Add Require Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

# Concat multiple file csv to one file

In [2]:
# from google.cloud import bigquery


# def make_query_utility(fuse_project_id, fuse_dataset_id, fuse_bucket, fuse_bq_key):
    
#     BQ_CLIENT = bigquery.Client.from_service_account_json(fuse_bq_key)


#     def query(query_string):
#         formatted_string = query_string.format(project=fuse_project_id, dataset=fuse_dataset_id)
#         query_job = BQ_CLIENT.query(formatted_string)
#         dataframe = query_job.result().to_dataframe().astype(float, errors='ignore')
#         return dataframe

#     return query

# query = make_query_utility('kohls-merch-in-lle', 'FUSE_STG01', 'kohls-merch-fuse-dataload', 'bigquery_key.json')


# data_test = query("""
# with sub as(
# select distinct subclass_id 
# from `kohls-merch-in-lle.FUSE_LOCAL_TEST.DEMAND_TRAINING_15_543_WALKIN`)
# , node as (select distinct node 
# from `kohls-merch-in-lle.FUSE_LOCAL_TEST.DEMAND_TRAINING_15_543_WALKIN`)
# ,start_week as
# (
# select DATE(DATETIME_SUB(DATETIME (DATE_TRUNC(CURRENT_DATE, WEEK(MONDAY))), INTERVAL 1 week)) as init_week
# ),
# future_16w as
# (
# select DATE(DATETIME_ADD(DATETIME (start_week.init_week), INTERVAL num1 WEEK)) as date_fut 
# from(select num1 
#        from UNNEST(GENERATE_ARRAY(1,16)) as num1
#     ),start_week
#       order by 1
# )
# ,hol_bow as(
#      select date_fut as bow, 
#             DATE_TRUNC(PARSE_DATE('%Y%m%d', CAST(hldy.Date_YYYYMMDD AS STRING)),week(MONDAY)) holiday
#        from future_16w ft
#        left join `kohls-merch-in-lle.HOLIDAYS.MAJOR_US_HOLIDAYS` hldy
#        on ft.date_fut=DATE_TRUNC(PARSE_DATE('%Y%m%d', CAST(hldy.Date_YYYYMMDD AS STRING)),week(MONDAY))
# )

# select subclass_id,node,hb.bow,
#         case when hb.holiday is null then '0' else '1' end holiday
#   from sub,node,hol_bow hb
# """)

In [3]:
# data.to_csv('train_data.csv', index=False)

In [2]:
data= pd.read_csv('test_preprocessing_Data.csv' , header=1 )
data.shape

(2450651, 5)

In [3]:
data.columns = ['index', 'subclass_id','node' , 'beginning_of_week' , 'is_holiday']
del data['index']

In [4]:
data.head().T

0                1                2  \
subclass_id               15-50-57         15-50-57         15-50-57   
node               {'node': '533'}  {'node': '533'}  {'node': '533'}   
beginning_of_week       2020-07-27       2020-09-21       2020-09-14   
is_holiday                       0                0                0   

                                 3                4  
subclass_id               15-50-57         15-50-57  
node               {'node': '533'}  {'node': '533'}  
beginning_of_week       2020-09-28       2020-06-29  
is_holiday                       0                1

# Convert node to number

In [5]:
data['node'] = data.node.astype(str)
data['node'] = data['node'].map(lambda x: ''.join([i for i in x if i.isdigit()]))
# data.node.value_counts().sort_index().to_csv('node.csv')
data.node = data.node.astype(int)

In [6]:
type(data.node.loc[1]) , data.node[1]

(numpy.int64, 533)

In [7]:
data['is_holiday'].value_counts().head(5)

0    1874027
1     576624
Name: is_holiday, dtype: int64

# prepare beginning_of_week

In [8]:
print("Min data from dataset:" , data['beginning_of_week'].min())
print("MAX data from dataset:" , data['beginning_of_week'].max())

Min data from dataset: 2020-06-29
MAX data from dataset: 2020-10-12


In [9]:
data['beginning_of_week'] = pd.to_datetime( data['beginning_of_week'] , format='%Y-%m-%d')

# prepare 'dept_nbr',  'dept_nbr',  'sub_cl_nbr' from 'subclass_id'

In [10]:
data['dept_nbr'] , data['cl_nbr'] , data['sub_cl_nbr'] = data['subclass_id'].str.split('-' , 2).str

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [11]:
data.head(5).T

0                    1  \
subclass_id                   15-50-57             15-50-57   
node                               533                  533   
beginning_of_week  2020-07-27 00:00:00  2020-09-21 00:00:00   
is_holiday                           0                    0   
dept_nbr                            15                   15   
cl_nbr                              50                   50   
sub_cl_nbr                          57                   57   

                                     2                    3  \
subclass_id                   15-50-57             15-50-57   
node                               533                  533   
beginning_of_week  2020-09-14 00:00:00  2020-09-28 00:00:00   
is_holiday                           0                    0   
dept_nbr                            15                   15   
cl_nbr                              50                   50   
sub_cl_nbr                          57                   57   

                                     4  
subclass_id                   15-50-57  
node                               533  
beginning_of_week  2020-06-29 00:00:00  
is_holiday                           1  
dept_nbr                            15  
cl_nbr                              50  
sub_cl_nbr                          57

# find advertised_purchases

In [12]:
data.isnull().sum()

subclass_id          0
node                 0
beginning_of_week    0
is_holiday           0
dept_nbr             0
cl_nbr               0
sub_cl_nbr           0
dtype: int64

In [13]:
data_train= pd.read_csv('train_data.csv')
data_train = data_train[[ 'beginning_of_week', 'total_revenue', 'advertised_purchases']]

In [14]:
type(data_train.beginning_of_week.loc[1])

str

In [15]:
import datetime
from datetime import date
start_day_last_year=datetime.datetime.now() - datetime.timedelta(days = 1*365)
start_day_last_year= start_day_last_year.strftime('%Y-%m-%d')
start_day_last_year

'2019-07-01'

In [16]:
mask = data_train['beginning_of_week'] > start_day_last_year 
data_train = data_train.loc[mask]
del data_train['beginning_of_week']
data_train= data_train.reset_index(drop=True)

In [17]:
data_train.shape

(7207800, 2)

In [18]:
data=pd.concat([data , data_train] ,axis=1)
data = data.dropna()
data.shape

(2450651, 9)

In [19]:
data.isnull().sum()

subclass_id             0
node                    0
beginning_of_week       0
is_holiday              0
dept_nbr                0
cl_nbr                  0
sub_cl_nbr              0
total_revenue           0
advertised_purchases    0
dtype: int64

# Feature Engineering for Time Series  Date-Related Features


In [20]:
data.node = data.node.astype(int)
data.is_holiday = data.is_holiday.astype(int)
data.dept_nbr = data.dept_nbr.astype(int)
data.cl_nbr = data.cl_nbr.astype(int)
data.sub_cl_nbr = data.sub_cl_nbr.astype(int)

# Read data from disk

In [21]:
data['is_holiday'].value_counts().head(5)

0    1874027
1     576624
Name: is_holiday, dtype: int64

# Convert beginning_of_week to readable for model

In [22]:
print("Min data from dataset:" , data['beginning_of_week'].min())
print("MAX data from dataset:" , data['beginning_of_week'].max())

Min data from dataset: 2020-06-29 00:00:00
MAX data from dataset: 2020-10-12 00:00:00


In [23]:
data['beginning_of_week'] = pd.to_datetime( data['beginning_of_week'] , format='%Y-%m-%d')

In [24]:
import datetime
import pandas as pd
# make the new columns using string indexing
data['year'] = data['beginning_of_week'].dt.year
data['month'] = data['beginning_of_week'].dt.month
data['day'] = data['beginning_of_week'].dt.day


In [25]:
data['quarter']=pd.to_datetime(data['beginning_of_week']).dt.quarter
data['smester']=np.where(data.quarter.isin([1,2]), 1 , 2)
data['day_of_year']=pd.to_datetime(data['beginning_of_week']).dt.dayofyear
data['week_of_year']=pd.to_datetime(data.beginning_of_week).dt.weekofyear
data['day_in_Month']=pd.to_datetime(data['beginning_of_week']).dt.days_in_month

In [26]:
data['week_of_year'].value_counts().head(6)

27    288312
42    144156
40    144156
39    144156
38    144156
37    144156
Name: week_of_year, dtype: int64

# CHECK MISSING VALUE

In [27]:
data.isnull().sum()

subclass_id             0
node                    0
beginning_of_week       0
is_holiday              0
dept_nbr                0
cl_nbr                  0
sub_cl_nbr              0
total_revenue           0
advertised_purchases    0
year                    0
month                   0
day                     0
quarter                 0
smester                 0
day_of_year             0
week_of_year            0
day_in_Month            0
dtype: int64

# channel Label Encoder

In [28]:
from sklearn import preprocessing 
def encoding(col):
    label_encod = preprocessing.LabelEncoder()
    data[col] = label_encod.fit_transform(data[col])
    return data[col].value_counts()

# encoding('channel')
encoding('day_of_year')
encoding('year')

    
# WALK-IN = 0
# webex_and_common = 1

0    2450651
Name: year, dtype: int64

# Convert subclass_id without dash on the data

In [29]:
# make string version of original column, call it 'col'
data['subclass_id'] = data['subclass_id'].str.replace(r'-', '').astype(int)
data['subclass_id'].head()

0    155057
1    155057
2    155057
3    155057
4    155057
Name: subclass_id, dtype: int64

In [99]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2450651 entries, 0 to 2450650
Data columns (total 17 columns):
 #   Column                Dtype         
---  ------                -----         
 0   subclass_id           int64         
 1   node                  int64         
 2   beginning_of_week     datetime64[ns]
 3   is_holiday            int64         
 4   dept_nbr              int64         
 5   cl_nbr                int64         
 6   sub_cl_nbr            int64         
 7   total_revenue         float64       
 8   advertised_purchases  float64       
 9   year                  int64         
 10  month                 int64         
 11  day                   int64         
 12  quarter               int64         
 13  smester               int64         
 14  day_of_year           int64         
 15  week_of_year          int64         
 16  day_in_Month          int64         
dtypes: datetime64[ns](1), float64(2), int64(14)
memory usage: 336.5 MB


# Add need column from data

In [30]:
# data_preprocess['total_revenue'] = data.total_revenue
# data_preprocess['advertised_purchases'] = data.advertised_purchases
data_preprocess = data

In [101]:
# del data_preprocess['total_revenue'], data_preprocess['advertised_purchases']

# Lag Features

In [31]:
lag_list = [1, 2 , 3]


for lag in lag_list:
    ft_name = ('advertise_shifted%s' % lag)
    data_preprocess[ft_name] = data['advertised_purchases'].shift(lag)
    # Fill the empty shifted features with 0
    data_preprocess[ft_name].fillna(0, inplace=True)
    
for lag in lag_list:
    ft_name = ('total_revenue_shifted%s' % lag)
    data_preprocess[ft_name] = data['total_revenue'].shift(lag)
    # Fill the empty shifted features with 0
    data_preprocess[ft_name].fillna(0, inplace=True)

# Rolling windows

In [32]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean]
function_name = ['min', 'max', 'mean']

for i in range(len(function_list)):
    data_preprocess[('advertised_purchases_rolling_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'total_revenue'])['advertised_purchases'].apply(function_list[i])
    
data_preprocess.shape 


(2450651, 26)

In [33]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']

for i in range(len(function_list)):
    data_preprocess[('total_revenue_rolling_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'advertised_purchases'])['total_revenue'].apply(function_list[i])
    
data_preprocess['total_revenue_rolling_std'].fillna(0 , inplace=True)

In [34]:
data.isnull().sum()

subclass_id                          0
node                                 0
beginning_of_week                    0
is_holiday                           0
dept_nbr                             0
cl_nbr                               0
sub_cl_nbr                           0
total_revenue                        0
advertised_purchases                 0
year                                 0
month                                0
day                                  0
quarter                              0
smester                              0
day_of_year                          0
week_of_year                         0
day_in_Month                         0
advertise_shifted1                   0
advertise_shifted2                   0
advertise_shifted3                   0
total_revenue_shifted1               0
total_revenue_shifted2               0
total_revenue_shifted3               0
advertised_purchases_rolling_min     0
advertised_purchases_rolling_max     0
advertised_purchases_roll

# Expanding Features

In [35]:
ft_name = ('expand_total_revenue_mean')
data_preprocess[ft_name] = data['total_revenue'].expanding(3).mean()
# Fill the empty shifted features with 0
data_preprocess[ft_name].fillna(0, inplace=True)
    
    
ft_name = ('expand_total_revenue_sum')
data_preprocess[ft_name] = data['total_revenue'].expanding(3).sum()
# Fill the empty shifted features with 0
data_preprocess[ft_name].fillna(0, inplace=True)



In [36]:
ft_name = ('expand_advertised_purchases_mean')
data_preprocess[ft_name] = data['advertised_purchases'].expanding(3).mean()
# Fill the empty shifted features with 0
data_preprocess[ft_name].fillna(0, inplace=True)
    
    
ft_name = ('expand_advertised_purchases_sum')
data_preprocess[ft_name] = data['advertised_purchases'].expanding(3).sum()
# Fill the empty shifted features with 0
data_preprocess[ft_name].fillna(0, inplace=True)



# Lag Features

In [37]:
lag_list = [1, 2]


for lag in lag_list:
    ft_name = ('is_holiday_shifted%s' % lag)
    data_preprocess[ft_name] = data['is_holiday'].shift(lag)
    # Fill the empty shifted features with 0
    data_preprocess[ft_name].fillna(0, inplace=True)

In [38]:
lag_list = [1, 2]


for lag in lag_list:
    ft_name = ('node_shifted%s' % lag)
    data_preprocess[ft_name] = data['node'].shift(lag)
    # Fill the empty shifted features with 0
    data_preprocess[ft_name].fillna(0, inplace=True)

In [39]:
data.isnull().sum()

subclass_id                          0
node                                 0
beginning_of_week                    0
is_holiday                           0
dept_nbr                             0
cl_nbr                               0
sub_cl_nbr                           0
total_revenue                        0
advertised_purchases                 0
year                                 0
month                                0
day                                  0
quarter                              0
smester                              0
day_of_year                          0
week_of_year                         0
day_in_Month                         0
advertise_shifted1                   0
advertise_shifted2                   0
advertise_shifted3                   0
total_revenue_shifted1               0
total_revenue_shifted2               0
total_revenue_shifted3               0
advertised_purchases_rolling_min     0
advertised_purchases_rolling_max     0
advertised_purchases_roll

# Rolling Window Feature

In [40]:
# Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=5, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=7, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=9, min_periods=1).std()

function_list = [f_min, f_max, f_mean]
function_name = ['min', 'max', 'mean']

for i in range(len(function_list)):
    data_preprocess[('node_new_%s' % function_name[i])] = data.sort_values('beginning_of_week').groupby([ 'sub_cl_nbr'])['node'].apply(function_list[i])
    
data_preprocess.shape 



(2450651, 41)

# Expanding Window Feature

In [41]:
data_preprocess['expand_node__sum'] =  data['node'].expanding(3).sum()
data_preprocess['expand_node__sum'].fillna(0, inplace=True)
    
data_preprocess['expand_node__mean'] =  data['node'].expanding(3).mean()
data_preprocess['expand_node__mean'].fillna(0, inplace=True)

# Remove unuse column 

In [42]:
del data_preprocess['subclass_id'] , data_preprocess['dept_nbr'], data_preprocess['cl_nbr'], data_preprocess['sub_cl_nbr']

In [43]:
data_preprocess.shape

(2450651, 39)

In [44]:
data_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2450651 entries, 0 to 2450650
Data columns (total 39 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   node                               int64         
 1   beginning_of_week                  datetime64[ns]
 2   is_holiday                         int64         
 3   total_revenue                      float64       
 4   advertised_purchases               float64       
 5   year                               int64         
 6   month                              int64         
 7   day                                int64         
 8   quarter                            int64         
 9   smester                            int64         
 10  day_of_year                        int64         
 11  week_of_year                       int64         
 12  day_in_Month                       int64         
 13  advertise_shifted1                 float64       
 14  ad

# Normalization the features

In [45]:
from sklearn.preprocessing import MinMaxScaler
def MIN_MAX_SCALE(col):
    X = data_preprocess[[col]].values # Modify to Numpy
    min_max_sc = preprocessing.MinMaxScaler()
    X_Scale = min_max_sc.fit_transform(X)
    data_preprocess[col] = pd.DataFrame(X_Scale) # add to pandas DB

In [46]:
list_of_max_min = [ 'total_revenue', 'advertised_purchases', 'node', 
                    'advertise_shifted1', 'advertise_shifted2', 'advertise_shifted3', 'total_revenue_shifted1', 
                   'total_revenue_shifted2', 'total_revenue_shifted3', 'advertised_purchases_rolling_min', 'advertised_purchases_rolling_max',
                   'advertised_purchases_rolling_mean', 'total_revenue_rolling_min', 'total_revenue_rolling_max', 'total_revenue_rolling_mean',
                   'total_revenue_rolling_std',
                   
                    'expand_total_revenue_mean', 'expand_total_revenue_sum', 'expand_advertised_purchases_mean', 
                   'expand_advertised_purchases_sum',
                   'node_shifted1', 'node_shifted2', 'node_new_min', 'node_new_max', 'node_new_mean', 'expand_node__sum', 'expand_node__mean']

for i in list_of_max_min:
    MIN_MAX_SCALE(i)

# Change columns name

# Reindex our data

In [47]:



col= ['total_revenue', 'advertised_purchases', 'is_holiday',
      'node', 'beginning_of_week', 'year', 'month', 'day', 'quarter', 'smester', 'day_of_year', 'week_of_year', 'day_in_Month', 
      
      'advertise_shifted1', 'advertise_shifted2', 'advertise_shifted3',
      
      'total_revenue_shifted1', 'total_revenue_shifted2', 'total_revenue_shifted3',
      'advertised_purchases_rolling_min','advertised_purchases_rolling_max', 'advertised_purchases_rolling_mean',
      'total_revenue_rolling_min', 'total_revenue_rolling_max', 'total_revenue_rolling_mean',
      'total_revenue_rolling_std',
      'expand_total_revenue_mean', 
      'expand_total_revenue_sum', 'expand_advertised_purchases_mean', 'expand_advertised_purchases_sum', 'is_holiday_shifted1',
      'is_holiday_shifted2', 'node_shifted1',
      'node_shifted2', 'node_new_min', 'node_new_max', 'node_new_mean', 'expand_node__sum', 'expand_node__mean' ]

data_preprocess = data_preprocess.reindex(columns=col)

In [48]:
print("Data Shape" , data_preprocess.shape)

Data Shape (2450651, 39)


In [50]:
# Change columns name to make sence for bigquery
data_preprocess.columns = ['total_revenue',
 'advertised_purchases',
 'is_holiday',
 'node',
 'beginning_of_week',
 'year',
 'month',
 'day',
 'quarter',
 'smester',
 'day_of_year',
 'week_of_year',
 'day_in_Month',
 'advertise_shifted_one',
 'advertise_shifted_two',
 'advertise_shifted_three',
 'total_revenue_shifted_one',
 'total_revenue_shifted_two',
 'total_revenue_shifted_three',
 'advertised_purchases_rolling_min',
 'advertised_purchases_rolling_max',
 'advertised_purchases_rolling_mean',
 'total_revenue_rolling_min',
 'total_revenue_rolling_max',
 'total_revenue_rolling_mean',
 'total_revenue_rolling_std',
 'expand_total_revenue_mean',
 'expand_total_revenue_sum',
 'expand_advertised_purchases_mean',
 'expand_advertised_purchases_sum',
 'is_holiday_shifted_one',
 'is_holiday_shifted_two',
 'node_shifted_one',
 'node_shifted_two',
 'node_new_min',
 'node_new_max',
 'node_new_mean',
 'expand_node__sum',
 'expand_node__mean']

# Create bigquery table

In [56]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "kohls-merch-in-lle.FUSE_LOCAL_TEST.PREDICTIONS_DEPT_15_543_test"




In [57]:
schema = [
bigquery.SchemaField("total_revenue", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("advertised_purchases", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("is_holiday", "INTEGER", mode="NULLABLE"),       
bigquery.SchemaField("node", "INTEGER", mode="NULLABLE"),       
bigquery.SchemaField("beginning_of_week", "TIMESTAMP", mode="NULLABLE"),
bigquery.SchemaField("year", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("month", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("day", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("quarter", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("smester", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("day_of_year", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("week_of_year", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("day_in_Month", "INTEGER", mode="NULLABLE"),         
bigquery.SchemaField("advertise_shifted_one", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("advertise_shifted_two", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("advertise_shifted_three", "FLOAT", mode="NULLABLE"),     
bigquery.SchemaField("total_revenue_shifted_one", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("total_revenue_shifted_two", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("total_revenue_shifted_three", "FLOAT", mode="NULLABLE"),   
bigquery.SchemaField("advertised_purchases_rolling_min", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("advertised_purchases_rolling_max", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("advertised_purchases_rolling_mean", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("total_revenue_rolling_min", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("total_revenue_rolling_max", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("total_revenue_rolling_mean", "FLOAT", mode="NULLABLE"),  
    bigquery.SchemaField("total_revenue_rolling_std", "FLOAT", mode="NULLABLE"),       

bigquery.SchemaField("expand_total_revenue_mean", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("expand_total_revenue_sum", "FLOAT", mode="NULLABLE"),   
bigquery.SchemaField("expand_advertised_purchases_mean", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("expand_advertised_purchases_sum", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("is_holiday_shifted_one", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("is_holiday_shifted_two", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("node_shifted_one", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("node_shifted_two", "FLOAT", mode="NULLABLE"),   
bigquery.SchemaField("node_new_min", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("node_new_max", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("node_new_mean", "FLOAT", mode="NULLABLE"),           
bigquery.SchemaField("expand_node__sum", "FLOAT", mode="NULLABLE"),       
bigquery.SchemaField("expand_node__mean", "FLOAT", mode="NULLABLE"),       
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table kohls-merch-in-lle.FUSE_LOCAL_TEST.PREDICTIONS_DEPT_15_543_test


In [59]:
data_preprocess.node = data_preprocess.node .astype(int)

In [60]:
data_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2450651 entries, 0 to 2450650
Data columns (total 39 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   total_revenue                      float64       
 1   advertised_purchases               float64       
 2   is_holiday                         int64         
 3   node                               int64         
 4   beginning_of_week                  datetime64[ns]
 5   year                               int64         
 6   month                              int64         
 7   day                                int64         
 8   quarter                            int64         
 9   smester                            int64         
 10  day_of_year                        int64         
 11  week_of_year                       int64         
 12  day_in_Month                       int64         
 13  advertise_shifted_one              float64       
 14  ad

# Write data to bigquery

In [61]:
import pandas_gbq
data_preprocess.to_gbq( destination_table = 'FUSE_LOCAL_TEST.PREDICTIONS_DEPT_15_543_test'  
                       ,project_id='kohls-merch-in-lle' ,if_exists='append'  ,chunksize=100000)

25it [11:54, 28.59s/it]


In [ ]:
"Done"

# Save data to bucket

In [75]:
# number_of_chunks = 15
# for id, df_i in  enumerate(np.array_split(data_preprocess, number_of_chunks)):
#     # the `id` inside {} may be omitted,
#     # I also inserted the missing closing parenthesis
#     df_i.to_csv('gs://kohls-merch-fuse-dataload/Preprocessed_data/data_pre_{}.csv'.format(id) , index=False)

# ADD Test data from bigquery